In [10]:
# try:
#     get_ipython().system('pip install pandas_datareader')
#     get_ipython().system('pip install pandas')
#     get_ipython().system('pip install matplotlib')
#     get_ipython().system('pip install requests')
#     get_ipython().system('pip install sympy')
#     get_ipython().system('pip install scipy')
#     get_ipython().system('pip install numpy')
#     get_ipython().system('pip install beautifulsoup4')
#     get_ipython().system('pip install numpy-financial')
# except Exception:
#     pass

nasdaq:GOLD 0.0 -0.38 -0.54 -0.061


In [11]:
import requests
import numpy_financial as npf
import numpy as np
from bs4 import BeautifulSoup
import threading
import re
import math


nasdaq:CLEU 0.15 0.044 0.073 0.0


In [12]:
tickers = []

with open('tickers.csv') as fh:
    for line in fh:
        row = line.strip()
        tickers.append(row)

In [13]:
def rdpModelReturn(payoutRatio, PE, returnOnEquity):
    return (1 + returnOnEquity*(1-payoutRatio))*(1+payoutRatio/PE)-1

In [14]:
# jitta temp mail: fesodiha@clout.wiki
# copy this following code to extract cookies from jitta.com and then manual replace number to string for some key, remove nested field
# document.cookie.split(';').map((c) => c.trim().split('=').map(decodeURIComponent)).reduce((a, b) => {try {a[b[0]] = JSON.parse(b[1]);}catch(e) {a[b[0]] = b[1];}return a;}, {});

cookies = {
    "_gid": "GA1.2.1787214795.1680600058",
    "ajs_anonymous_id": "ba83fd06-acd4-491c-87f0-93d5a0ec1167",
    "_fbp": "fb.1.1680600058107.1290303553",
    "gclid": "undefined",
    "hunt7586-_zldp": "ss+YrC93NjOVwgyi/EREBDDyggBwhhrsjOe1mSQ5qB4Gl/cFmC1zp9ADOaZpPlyn7uRfjfIQJjw=",
    "hunt7586-_zldt": "cffe8506-b037-4727-b0fe-0df85ace2121-2",
    "JDCID": "efbbd5c32d34555ef7b4bbcbc9a0af863da8f46f65c27f7c67b4e5e9a5b97cfa.5439c914472955e6ecc4ac72cbf4f297dafa500b94dc38a5e2b63278cc6c5369275321157f5bbd49210a4b2cc4dec3851703061b2613e31a45162e61f84910a362b436eb4058a50cb920a0f8f433b0620c295848890c",
    "_ga": "GA1.1.1731066012.1680600058",
    "_ga_BT0RBD348J": "GS1.1.1680600057.1.1.1680600136.44.0.0",
    "ajs_user_id": "642bec1b2e042000174a3d46",
    "jitta_nu": "2"
}

In [15]:
def convert_row_to_data(row):
    r = row.find_all('span')
    c = map(lambda x: p2f(x.text), r)
    return list(c)[:-2]

def p2f(x):
    cx = x.replace('%', '').replace(',', '')
    return None if cx == '- -' or cx == '?' else float(cx)

def rate_of_change(nums):
    return list(map(lambda i: None if nums[i-1] is None or nums[i] is None else nums[i] / nums[i-1] - 1, range(1, len(nums))))

def mean(nums):
    nums = list(filter(lambda x: x is not None, nums))
    return 0.0 if len(nums) == 0 else sum(nums) / len(nums)

def median(nums):
    nums = sorted(filter(lambda x: x is not None, nums))
    n = len(nums)
    if n == 0:
        return 0.0
    if n % 2 == 1:
        return nums[n // 2]
    else:
        return (nums[n // 2 - 1] + nums[n // 2]) / 2


def clean_data(y):
    n = len(y)
    if n == 0:
        return []
    cleaned_y = [0.0] * n
    cleaned_y[0] = y[0] if y[0] is not None else 0.0
    cleaned_y[-1] = y[-1] if y[-1] is not None else 0.0
    for i in range(1, n-1):
        if y[i] is None:
            cleaned_y[i] = 0.0
        elif abs(y[i] - cleaned_y[i-1]) > abs(y[i] - y[i+1]):
            cleaned_y[i] = y[i+1]
        else:
            cleaned_y[i] = y[i-1]
    return cleaned_y


def linear_regression(y):
    cleaned_y = clean_data(y)
    n = len(cleaned_y)
    x = list(range(n))
    x_mean = sum(x) / n
    y_mean = sum(cleaned_y) / n
    cov = sum([(x[i] - x_mean) * (cleaned_y[i] - y_mean) for i in range(n)])
    var = sum([(x[i] - x_mean) ** 2 for i in range(n)])
    slope = cov / var
    intercept = y_mean - slope * x_mean
    y_pred = [slope * i + intercept for i in x]
    roc = 0 if y_mean == 0 else slope / y_mean
    return x, slope, intercept, y_pred, roc, y_mean

def dcf_irr(price, eps, g, dpr):
    tax_rate = 0.15
    adj_dpr = (1 + dpr * (1 - tax_rate))
    cashflows = [price * -1]
    g1 = g
    g2 = 0.5 * g
    g3 = min(0.10, 0.2 * g)
    g4 = min(0.04, 0.1 * g)
    last_eps = eps
    
    for i in range(0, 40):
        if i < 5:
            last_eps = last_eps * (1 + g1)
        elif i < 10:
            last_eps = last_eps * (1 + g2)
        elif i < 20:
            last_eps = last_eps * (1 + g3)
        else:
            last_eps = last_eps * (1 + g4)

        cashflows.append(last_eps * adj_dpr)
    irr = npf.irr(cashflows)
    return 0.0 if np.isnan(irr) else irr

In [16]:
def calculate(ticker):
    with requests.post('https://www.jitta.com/stock/' + ticker + '/factsheet', cookies=cookies) as r:
        soup = BeautifulSoup(r.content, 'lxml')

        try:
            price = float(re.search(
                r"\.price\.latest\":{\"datetime\":\"(.*)\",\"close\":(.*),\"market_cap\"", r.content.decode('utf-8'))[2])

            row = soup.find_all('div', {'class': 'dAGwCX'})

            if len(row) == 0:
                return None
        except:
            return None

        REV = convert_row_to_data(row[0])
        EPS = convert_row_to_data(row[1])
        ROE = convert_row_to_data(row[20])
        ROA = convert_row_to_data(row[21])
        DE = convert_row_to_data(row[23])
        DPR = convert_row_to_data(row[24])

        eps_forecast = linear_regression(
            list(map(lambda x: 0 if x == None else x, EPS)))
        dpr_forecast = linear_regression(
            list(map(lambda x: 0 if x == None else max(0, min(100, x)), DPR)))

        eps = float(eps_forecast[3][-1])
        eps_pos = len(list(filter(lambda x: x is not None and x > 0, EPS)))
        g = float(eps_forecast[4])
        dpr = float(dpr_forecast[3][-1] / 100)

        irr = float(dcf_irr(price, eps, g, dpr))

        print(f"{ticker} {irr:.2} {g:.2} {eps:.2} {dpr:.2}")

        return {
            "TICKER": ticker,
            "PRICE": price,
            "EPS": eps,
            "EPS_POS": eps_pos,
            "EPS_N": len(EPS),
            "G": g,
            "DPR": dpr,
            "IRR": irr,
        }


In [17]:
def run_fetch_stock(tickers, thread_no):
  f = open(f"data/stock-screen-{thread_no}.csv", 'w')
  f.write('ticker|IRR|G|EPS|EPS_POS|EPS_N|DPR|PRICE\n')
  for ticker in tickers:
    # print(ticker)
    fetch_stock(ticker, f)
    
  f.close()
  

def fetch_stock(ticker, f):
    data = calculate(ticker)

    if data == None:
        return

    f.write(ticker + '|' +
            '{:.4f}'.format(data['IRR']) + '|' +
            '{:.4f}'.format(data['G']) + '|' +
            '{:.4f}'.format(data['EPS']) + '|' +
            '{:.4f}'.format(data['EPS_POS']) + '|' +
            '{:.4f}'.format(data['EPS_N']) + '|' +
            '{:.4f}'.format(data['DPR']) + '|' +
            '{:.4f}'.format(data['PRICE']) + '\n'
            )


In [18]:
threads  = []
total_thread = 50
split_tickers = np.array_split(tickers, total_thread)

for i in range(0, total_thread):
    thread = threading.Thread(target=run_fetch_stock, args=(split_tickers[i], i,))
    threads.append(thread)
    thread.start()

for thread in threads:
    thread.join()


nasdaq:SCPL 0.23 0.29 1.0 0.0
nasdaq:MTC 0.0 0.2 -1.1 0.0
nasdaq:CDTX 0.0 -0.14 -0.9 0.0
nasdaq:WTW 0.083 0.094 8.4 0.36
nasdaq:MATW 0.0 2.9e+01 -3.0 -0.034
nasdaq:APEN 0.0 -0.18 -0.73 0.0
nasdaq:AEI 0.0 0.039 -7.9 0.0
nasdaq:SCRM 0.19 0.6 0.091 0.0
nasdaq:OBLG 0.0 0.32 -1.7e+01 0.0
nasdaq:TERN 0.0 0.26 -2.7 0.0
nasdaq:LKFN 0.16 0.11 4.2 0.39
nasdaq:KURA 0.0 0.087 -2.1 0.0
nasdaq:IEP -0.19 -0.25 0.042 0.049
nasdaq:AZPN 0.0058 0.061 2.8 0.0
nasdaq:CLFD 0.17 0.3 1.6 0.0
nasdaq:ALIM 0.44 -0.39 2.7 0.0
nasdaq:GNSS 0.0 2.3 -0.1 0.0
nasdaq:SISI 0.0 -0.83 -2.6 0.0
nasdaq:FRME 0.25 0.11 4.0 0.35
nasdaq:REKR 0.0 0.18 -0.55 0.0
nasdaq:DENN 0.27 0.18 1.4 0.0
nasdaq:SUPN 0.13 0.16 1.9 0.0
nasdaq:MTCH 0.046 0.08 1.3 -0.1
nasdaq:GECC 0.0 0.27 -5.2 0.0
nasdaq:WALD 0.0 0.0 0.0 0.0
nasdaq:PNBK -0.086 -0.2 0.19 0.0021
nasdaq:SOTK 0.081 0.22 0.096 0.0
nasdaq:ELOX 0.0 -0.052 -4.1e+01 0.0
nasdaq:MIND 0.0 0.063 -1.7 0.0
nasdaq:TESS 0.0 5.2 -1.6 -0.0098
nasdaq:ASTR 0.0 0.4 -1.5 0.0
nasdaq:ROST 0.11 0.12 5.0 

KeyboardInterrupt: 

nasdaq:RELY 0.0 0.44 -0.6 0.0
nasdaq:APLD 0.0 0.53 -0.27 0.0
nasdaq:CTRN 0.51 0.31 3.7 0.079
nasdaq:ELTK -0.14 -0.5 0.67 0.12
nasdaq:MSGM 0.0 0.52 -2.1e+01 0.0
nasdaq:MAYS -0.19 -0.24 0.031 0.0
nasdaq:HBNC 0.37 0.14 2.1 0.3
nasdaq:RPHM 0.0 0.45 -1.7 0.0
nasdaq:OTLY 0.0 0.49 -0.49 0.0
nasdaq:FGBI 0.27 0.088 2.2 0.3
nasdaq:PEP 0.086 0.061 6.6 0.79
nasdaq:AEMD 9.4 -0.38 6.1 0.0
nasdaq:SJ 0.46 0.3 0.75 0.0
nasdaq:NRIX 0.0 0.46 -3.0 0.0
nasdaq:CLIR 0.0 -0.21 -0.085 0.0
nasdaq:OCC 0.0 0.056 -0.37 -0.12
nasdaq:DFLI 0.0 0.96 -0.097 0.0
nasdaq:UK 0.0 0.3 -2.2e+01 0.0
nasdaq:TRIP -0.41 -1.1 -1.1 0.0
nasdaq:NEOV 0.0 0.43 -0.24 0.0
nasdaq:WAVD 0.0 -0.031 -0.0066 0.0
nasdaq:GNUS 0.0 -0.22 -1.3 0.0
nasdaq:IFRX 0.0 0.14 -1.7 0.0
nasdaq:LMB 0.061 0.13 0.48 0.0
nasdaq:FRSH 0.0 0.43 -2.8 0.0
nasdaq:MTEX -0.12 -0.21 0.14 0.13
nasdaq:IQ -0.053 -0.29 0.93 0.0
nasdaq:PULM 5.5 -0.29 2.4e+01 0.0
nasdaq:SVC -0.37 -1.7 -1.3 0.067
nasdaq:VIAV 0.0 -1.8 0.25 0.0
nasdaq:RENE 0.18 0.6 0.091 0.0
nasdaq:HBT 0.25 0.11 